## Шаг 1: Загрузка данных

Загружаем все необходимые данные для решения задачи:
- **data_heroes.csv**: информация о героях и их запасах очков хода
- **dist_objects.csv**: матрица расстояний между объектами (700x700)
- **dist_start.csv**: расстояния от замка (депо) до каждого объекта
- **data_objects.csv**: информация об объектах (день появления и награда)

# Бейзлайн решение задачи "Оптимизация Маршрутизации Героев" (ОМГ)

## Описание задачи

Задача представляет собой вариант **VRPTW (Vehicle Routing Problem with Time Windows)** - задачу маршрутизации транспортных средств с временными окнами. В нашем случае:
- **Депо** = замок с таверной
- **Транспортные средства** = герои (с разными запасами очков хода)
- **Заказы** = водяные мельницы (объекты)
- **Временные окна** = дни доступности мельниц (day_open от 1 до 7)
- **Матрица расстояний** = матрица очков хода между объектами

## Ключевые особенности задачи

1. **Гетерогенный парк героев**: у разных героев разный запас ежедневных очков хода (от 1300 до 1900)
2. **Глобальный счетчик дней**: все герои синхронизированы по дням, очки хода восполняются в начале каждого дня
3. **Правила таверны**: герои нанимаются последовательно по hero_id (чтобы нанять героя N, нужно нанять всех от 1 до N-1)
4. **Правило "последнего шага"**: если у героя осталось от 1 до 100 очков хода и не хватает на посещение, посещение все равно считается успешным
5. **Ожидание на месте**: если герой прибыл к мельнице раньше дня её появления, он ждет на месте до нужного дня

## Стратегия решения

**Основная идея**: решаем задачу итеративно по дням - так не придется разбираться с временными окнами.

**Алгоритм**:
1. **День 1**: Решаем классическую VRP задачу для всех объектов с `day_open == 1`, используя 20 героев (потому что я подобрала, что 20 - это неплохо). Все герои стартуют из замка.
2. **Дни 2-7**: Для каждого героя продолжаем маршрут с его конечной точки предыдущего дня. Если герой ничего не делал в предыдущий день, то он точно может дойти до любого объекта (можно проверить по матрице расстояний) и тогда считаем, что расстояние до всех мельниц 0 от конечной точки предыдущего дня.


In [1]:
# ============================================================================
# ЗАГРУЗКА ДАННЫХ
# ============================================================================

# Импорт необходимых библиотек
import pandas as pd

# Данные о героях: содержит hero_id (1-100) и move_points (дневной запас очков хода)
# Всего доступно 100 героев с разными запасами очков хода (от 1300 до 1900)
df_heroes = pd.read_csv('data_heroes.csv')

# Матрица расстояний между объектами: 700x700 матрица очков хода между мельницами
# Индексы строк и столбцов соответствуют object_id (от 1 до 700)
df_dist = pd.read_csv('dist_objects.csv')

# Расстояния от замка (депо) до каждого объекта: 700 строк с расстояниями от замка
# Используется для построения начальных маршрутов героев
df_dist_start = pd.read_csv('dist_start.csv')

# Данные об объектах: содержит object_id, day_open (день появления 1-7) и reward (награда)
# Всего 700 объектов, награда для всех равна 500
df_day_open = pd.read_csv('data_objects.csv')

# ============================================================================
# ГЛОБАЛЬНЫЕ ПАРАМЕТРЫ ЗАДАЧИ
# ============================================================================

visit_cost = 100   # Стоимость посещения мельницы в очках хода (аналог времени обслуживания)
hero_cost = 2500   # Стоимость найма одного героя в таверне
reward = 500       # Награда за успешное посещение мельницы (для всех объектов одинаковая)

# Выводим статистику загруженных данных
print("=" * 60)
print("СТАТИСТИКА ЗАГРУЖЕННЫХ ДАННЫХ")
print("=" * 60)
print(f"Загружено героев: {len(df_heroes)}")
print(f"Загружено объектов: {len(df_day_open)}")
print(f"\nРаспределение объектов по дням появления:")
print(df_day_open['day_open'].value_counts().sort_index())
print(f"\nДиапазон очков хода героев: {df_heroes['move_points'].min()} - {df_heroes['move_points'].max()}")
print(f"Медиана очков хода: {df_heroes['move_points'].median()}")
print("=" * 60)

СТАТИСТИКА ЗАГРУЖЕННЫХ ДАННЫХ
Загружено героев: 100
Загружено объектов: 700

Распределение объектов по дням появления:
day_open
1    134
2    111
3    121
4    105
5     63
6     98
7     68
Name: count, dtype: int64

Диапазон очков хода героев: 1500 - 1900
Медиана очков хода: 1560.0


## Шаг 2: Построение полной матрицы расстояний

Для решения задачи VRP нам нужна полная матрица расстояний, включающая:
- Расстояния между всеми объектами (мельницами)
- Расстояния от депо (замка) до всех объектов
- Расстояния от всех объектов до депо

**Важно**: Депо имеет индекс 0 в матрице и является стартовой точкой для всех героев в первый день.

In [2]:
# ============================================================================
# ПОСТРОЕНИЕ ПОЛНОЙ МАТРИЦЫ РАССТОЯНИЙ
# ============================================================================

# Копируем матрицу расстояний между объектами
df = df_dist.copy()

# Получаем расстояния от депо (замка) до каждого объекта
# Это будет использоваться как первый столбец и первая строка матрицы
depot_dist = df_dist_start['dist_start'].values

# Добавляем столбец расстояний от депо (расстояния от депо до каждого объекта)
df.insert(0, 'depot', depot_dist)

# Добавляем строку для депо: [0, dist_to_obj1, dist_to_obj2, ...]
# Первый элемент 0 - расстояние от депо до самого депо
depot_row = [0] + list(depot_dist)
df.loc['depot'] = depot_row

# Переупорядочиваем индексы так, чтобы депо был первым
# Это важно для OR-Tools, где депо должен иметь индекс 0
df = df.reindex(['depot'] + df.index[:-1].tolist())

print(f"✓ Матрица расстояний создана: {df.shape[0]}x{df.shape[1]}")
print(f"✓ Депо имеет индекс: '{df.index[0]}' (будет преобразован в 0 в матрице)")
print(f"✓ Матрица содержит {df.shape[0]-1} объектов + 1 депо")


✓ Матрица расстояний создана: 701x701
✓ Депо имеет индекс: 'depot' (будет преобразован в 0 в матрице)
✓ Матрица содержит 700 объектов + 1 депо


In [3]:
df

,depot,object_1,object_2,object_3,object_4,object_5,object_6,object_7,object_8,object_9,...,object_691,object_692,object_693,object_694,object_695,object_696,object_697,object_698,object_699,object_700
depot,0,413,373,386,93,466,480,240,440,773,...,426,493,453,426,666,720,693,600,480,120
0,413,0,300,347,467,353,353,360,260,507,...,173,207,433,367,453,507,460,407,520,480
1,373,300,0,207,447,233,280,380,213,613,...,367,347,233,493,453,513,467,400,327,453
2,386,347,207,0,440,367,407,360,347,747,...,393,393,360,500,587,647,600,533,447,447
3,93,467,447,440,0,547,587,287,513,867,...,520,527,527,473,780,840,787,733,560,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,720,507,513,647,840,473,413,760,407,413,...,567,600,647,760,240,0,233,187,747,853
696,693,460,467,600,787,427,373,713,353,280,...,527,553,573,700,233,233,0,160,653,800
697,600,407,400,533,733,360,307,653,300,340,...,467,500,520,653,80,187,160,0,647,747
698,480,520,327,447,560,467,500,547,407,833,...,573,573,267,680,693,747,653,647,0,587


## Шаг 3: Решение VRP для первого дня

### Стратегия для дня 1

В первый день все объекты с `day_open == 1` доступны одновременно, поэтому мы можем решить классическую VRP задачу без учета временных окон. Это упрощает решение, так как:
- Все герои стартуют из одного места (замок/депо)
- Нет необходимости учитывать ожидание на месте
- Можно использовать стандартные алгоритмы VRP

**Подход**: Используем библиотеку OR-Tools для решения задачи VRP с ограничениями по емкости (очкам хода героев).

In [4]:
# ============================================================================
# ФИЛЬТРАЦИЯ ОБЪЕКТОВ ПЕРВОГО ДНЯ
# ============================================================================

# Получаем список object_id всех объектов, доступных в первый день
open_idx = df_day_open.loc[df_day_open['day_open'] == 1, 'object_id']

# Формируем список колонок для подматрицы: депо + все объекты первого дня
cols = ['depot'] + [f'object_{i}' for i in list(open_idx)]

# Создаем подматрицу расстояний только для объектов первого дня
# Индексы объектов в df: object_id - 1 (так как object_id начинается с 1)
df_filtered = df.loc[['depot'] + list(open_idx - 1), cols]

print(f"✓ Объектов первого дня: {len(open_idx)}")
print(f"✓ Размер подматрицы: {df_filtered.shape[0]}x{df_filtered.shape[1]}")
print(f"  (включая депо и {len(open_idx)} объектов)")

✓ Объектов первого дня: 134
✓ Размер подматрицы: 135x135
  (включая депо и 134 объектов)


In [5]:
# Преобразуем DataFrame в список списков (матрицу) для использования в OR-Tools
# OR-Tools требует матрицу в формате list[list[int]]
matrix = df_filtered.to_numpy().tolist()

### Добавление стоимости посещения к матрице расстояний

**Важно**: При переходе от одного объекта к другому нужно учесть не только расстояние, но и стоимость посещения целевого объекта.

**Логика**:
- Переход от объекта A к объекту B = расстояние(A, B) + visit_cost
- Это моделирует ситуацию, когда герой тратит очки хода на перемещение и на посещение мельницы
- Возврат в депо не требует затрат (герой на самом деле не возвращается в замок)

In [6]:
# ============================================================================
# МОДИФИКАЦИЯ МАТРИЦЫ: ДОБАВЛЕНИЕ СТОИМОСТИ ПОСЕЩЕНИЯ
# ============================================================================

n = len(matrix)
depot = 0

# Добавляем visit_cost ко всем переходам между узлами
# Это моделирует затраты на перемещение + посещение целевого объекта
# Возврат в депо не требует затрат на посещение (на самом деле вообще туда не возвращаемся)
for i in range(n):
    for j in range(len(matrix[i])):
        if i != j:  # Не добавляем стоимость для перехода в самого себя
            if j == depot:
                # Возврат в депо не требует затрат на посещение
                # Устанавливаем сразу, чтобы не добавлять visit_cost
                matrix[i][depot] = 0
            else:
                # Добавляем стоимость посещения для переходов к объектам
                matrix[i][j] += visit_cost

print(f"✓ Стоимость посещения ({visit_cost} очков) добавлена ко всем переходам")
print(f"✓ Возврат в депо установлен в 0 очков (оптимизировано: устанавливается сразу)")

✓ Стоимость посещения (100 очков) добавлена ко всем переходам
✓ Возврат в депо установлен в 0 очков (оптимизировано: устанавливается сразу)


### Настройка параметров героев

**Правило "последнего шага" (ласт-мув)**: Если у героя осталось от 1 до 100 очков хода и ему не хватает всех 100 очков для посещения мельницы, посещение все равно считается успешным.

**Реализация**: Добавляем 100 очков к запасу каждого героя, чтобы учесть возможность использования последних очков хода. Это позволяет герою "дотянуться" до мельницы даже если у него осталось мало очков.


In [7]:
# ============================================================================
# НАСТРОЙКА ПАРАМЕТРОВ ГЕРОЕВ
# ============================================================================

# Начальное количество героев для первого дня
N = 20

# Добавляем 100 очков к запасу каждого героя для учета правила "последнего шага"
# Это позволяет герою использовать последние очки хода для посещения мельницы
max_distances = list(df_heroes['move_points'][:N] + 100)

print(f"✓ Используем {N} героев для первого дня")
print(f"✓ Диапазон очков хода (без учета ласт-мув): {min(max_distances)-100} - {max(max_distances)-100}")
print(f"✓ Диапазон очков хода (с учетом ласт-мув): {min(max_distances)} - {max(max_distances)}")
print(f"✓ Медиана очков хода: {df_heroes['move_points'][:N].median()}")

✓ Используем 20 героев для первого дня
✓ Диапазон очков хода (без учета ласт-мув): 1560 - 1900
✓ Диапазон очков хода (с учетом ласт-мув): 1660 - 2000
✓ Медиана очков хода: 1560.0


In [8]:
# Импорт библиотеки OR-Tools для решения VRP
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [9]:
## Шаг 4: Решение VRP задачи для первого дня

### Использование OR-Tools

# OR-Tools (Google Optimization Tools) - библиотека для решения задач оптимизации, включая VRP.

def create_data_model():
    """Создает модель данных для задачи VRP."""
    data = {}
    data["distance_matrix"] = matrix
    data["num_vehicles"] = N
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Выводит решение на экран и возвращает список маршрутов."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    routes = []
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route = []
        
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        
        routes.append(route)
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    
    print(f"Maximum of the route distances: {max_route_distance}m")
    return routes


def solve_vrp_day1():
    """Решает VRP задачу для первого дня."""
    # Создаем модель данных
    data = create_data_model()

    # Создаем менеджер индексов маршрутизации
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Создаем модель маршрутизации
    routing = pywrapcp.RoutingModel(manager)

    # Создаем callback для расстояний
    def distance_callback(from_index, to_index):
        """Возвращает расстояние между двумя узлами."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    PENALTY = 50000  # подобрать значение
    
    for node in range(1, len(data["distance_matrix"])):  # 0 — депо, его не трогаем
        index = manager.NodeToIndex(node)
        routing.AddDisjunction([index], PENALTY)

    # Добавляем ограничение по расстоянию (емкость)
    dimension_name = "Distance"
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        max_distances,  # максимальное расстояние для каждого транспортного средства
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)

    # Настройка параметров поиска решения
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 120
    
    # Решаем задачу
    solution = routing.SolveWithParameters(search_parameters)

    # Выводим решение
    if solution:
        routes = print_solution(data, manager, routing, solution)
        return routes
    else:
        print("No solution found !")
        return []


# Решаем задачу для первого дня
print("=" * 60)
print("РЕШЕНИЕ ДЛЯ ДНЯ 1")
print("=" * 60)
routes_day1 = solve_vrp_day1()

РЕШЕНИЕ ДЛЯ ДНЯ 1
Objective: 632680
Route for vehicle 0:
 0 ->  98 ->  109 ->  119 ->  120 ->  12 ->  4 -> 0
Distance of the route: 1779m

Route for vehicle 1:
 0 ->  59 ->  36 ->  39 ->  75 ->  78 ->  57 ->  53 -> 0
Distance of the route: 1658m

Route for vehicle 2:
 0 ->  20 ->  5 ->  37 ->  92 ->  8 ->  38 ->  10 ->  9 ->  6 -> 0
Distance of the route: 1795m

Route for vehicle 3:
 0 ->  117 ->  31 ->  124 ->  58 ->  73 ->  62 -> 0
Distance of the route: 1687m

Route for vehicle 4:
 0 ->  45 ->  55 ->  15 ->  44 ->  110 ->  40 -> 0
Distance of the route: 1600m

Route for vehicle 5:
 0 ->  82 ->  71 ->  60 ->  96 ->  106 -> 0
Distance of the route: 1573m

Route for vehicle 6:
 0 ->  86 ->  116 ->  125 ->  13 ->  81 -> 0
Distance of the route: 1353m

Route for vehicle 7:
 0 ->  85 ->  115 ->  7 ->  11 ->  100 -> 0
Distance of the route: 1553m

Route for vehicle 8:
 0 ->  94 ->  132 ->  1 ->  51 ->  50 ->  46 ->  17 -> 0
Distance of the route: 1643m

Route for vehicle 9:
 0 ->  21 ->  8

In [10]:
# После первого дня фильтруем до активных героев (удаляем тех, кто ничего не делал)
# Эти активные герои будут использоваться на ВСЕ последующие дни
routes_day1 = [route for route in routes_day1 if route != [0]]
num_active_heroes = len(routes_day1)

print(f"\nАктивных героев после фильтрации: {num_active_heroes}")
print(f"Используем героев с ID от 1 до {num_active_heroes}")
print(f"ВАЖНО: Эти {num_active_heroes} героев остаются активными на ВСЕ дни (2-7)")
print(f"       Если герой ничего не делает в день N, он остается в списке с пустым маршрутом,")
print(f"       но его позиция сохраняется из предыдущего дня.")

# Сохраняем информацию о конечных точках всех активных героев для последующих дней
# Маршруты дня 1 заданы индексами подматрицы (1 = первый объект с day_open==1, 2 = второй, ...)
# Преобразуем индекс в реальный object_id по списку объектов первого дня
open_idx_day1 = df_day_open.loc[df_day_open['day_open'] == 1, 'object_id'].values
hero_end_positions = {}
for hero_idx, route in enumerate(routes_day1):
    last_node = 0  # индекс в подматрице дня 1 (0 = депо, 1..n = объекты)
    for node in reversed(route):
        if node != 0:
            last_node = node
            break
    # Индекс 1 в маршруте = первый объект в open_idx_day1, т.е. open_idx_day1[0]
    hero_end_positions[hero_idx] = open_idx_day1[last_node - 1] if last_node > 0 else 0

print(f"\nКонечные позиции героев после дня 1:")
for hero_idx, pos in hero_end_positions.items():
    print(f"  Герой {hero_idx + 1}: объект {pos if pos > 0 else 'депо'}")


Активных героев после фильтрации: 20
Используем героев с ID от 1 до 20
ВАЖНО: Эти 20 героев остаются активными на ВСЕ дни (2-7)
       Если герой ничего не делает в день N, он остается в списке с пустым маршрутом,
       но его позиция сохраняется из предыдущего дня.

Конечные позиции героев после дня 1:
  Герой 1: объект 14
  Герой 2: объект 271
  Герой 3: объект 24
  Герой 4: объект 335
  Герой 5: объект 202
  Герой 6: объект 573
  Герой 7: объект 422
  Герой 8: объект 514
  Герой 9: объект 75
  Герой 10: объект 339
  Герой 11: объект 112
  Герой 12: объект 405
  Герой 13: объект 612
  Герой 14: объект 570
  Герой 15: объект 241
  Герой 16: объект 163
  Герой 17: объект 121
  Герой 18: объект 588
  Герой 19: объект 420
  Герой 20: объект 610


## Дни 2-7: Продолжение маршрутов

Для каждого героя продолжаем маршрут с его конечной точки предыдущего дня.
Создаем индивидуальные матрицы расстояний для каждого героя, где депо - это его конечная точка предыдущего дня.

### Вспомогательные функции для решения дней 2-7

In [11]:
# Преобразуем матрицу расстояний в список для быстрого доступа
total_dists = df_dist.to_numpy().tolist()


def get_hero_end_position(hero_idx, route, hero_end_positions):
    """Определяет конечную позицию героя для начала следующего дня."""
    if len(route) > 1:
        for node in reversed(route):
            if node != 0:
                return node
    return hero_end_positions.get(hero_idx, 0)


def create_vehicle_matrix_for_day(day, hero_end_positions, previous_routes_empty=None):
    """Создает матрицу расстояний для конкретного дня и всех героев."""
    vehicle_distance_matrices = []
    previous_routes_empty = previous_routes_empty or {}
    for hero_idx in range(len(hero_end_positions)):
        end_pos = hero_end_positions[hero_idx]
        stood_still = previous_routes_empty.get(hero_idx, False)
        if stood_still:
            depot_dist = [0] * 700
        elif end_pos == 0:
            depot_dist = df_dist_start['dist_start'].values
        else:
            depot_dist = [total_dists[end_pos - 1][i] for i in range(700)]
        df_vehicle = df_dist.copy()
        df_vehicle.insert(0, 'depot', depot_dist)
        depot_row = [0] + list(depot_dist)
        df_vehicle.loc['depot'] = depot_row
        df_vehicle = df_vehicle.reindex(['depot'] + df_vehicle.index[:-1].tolist())
        open_idx = df_day_open.loc[df_day_open['day_open'] == day, 'object_id'].sort_values()
        cols = ['depot'] + [f'object_{i}' for i in list(open_idx)]
        df_filtered = df_vehicle.loc[['depot'] + list(open_idx - 1), cols]
        matrix = df_filtered.to_numpy().tolist()
        n = len(matrix)
        depot = 0
        for i in range(n):
            for j in range(len(matrix[i])):
                if i != j:
                    if j == depot:
                        matrix[i][depot] = 0
                    else:
                        matrix[i][j] += visit_cost
        vehicle_distance_matrices.append(matrix)
    return vehicle_distance_matrices


def solve_vrp_day_n(day, hero_end_positions, previous_routes=None):
    """Решает VRP задачу для дня N (2-7)."""
    num_heroes = len(hero_end_positions)
    previous_routes_empty = {}
    if previous_routes is not None:
        for hero_idx, route in enumerate(previous_routes):
            previous_routes_empty[hero_idx] = len(route) <= 1
    vehicle_distance_matrices = create_vehicle_matrix_for_day(day, hero_end_positions, previous_routes_empty)
    matrix_size = len(vehicle_distance_matrices[0])
    data = {"distance_matrix": vehicle_distance_matrices[0], "num_vehicles": num_heroes, "depot": 0}
    manager = pywrapcp.RoutingIndexManager(matrix_size, data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)
    def make_distance_callback(vehicle_id):
        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return vehicle_distance_matrices[vehicle_id][from_node][to_node]
        return distance_callback
    transit_callback_indices = []
    for vehicle_id in range(data["num_vehicles"]):
        callback = make_distance_callback(vehicle_id)
        callback_index = routing.RegisterTransitCallback(callback)
        routing.SetArcCostEvaluatorOfVehicle(callback_index, vehicle_id)
        transit_callback_indices.append(callback_index)
    VEHICLE_FIXED_COST = 100000 # лучше меньше героев, чем больше героев
    for vehicle_id in range(data["num_vehicles"]):
        routing.SetFixedCostOfVehicle(VEHICLE_FIXED_COST, vehicle_id)
    dimension_name = "Distance"
    max_distances_day = list(df_heroes['move_points'][:num_heroes] + 100)
    routing.AddDimensionWithVehicleTransitAndCapacity(transit_callback_indices, 0, max_distances_day, True, dimension_name)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 10
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        routes = []
        for vehicle_id in range(data["num_vehicles"]):
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            routes.append(route)
        # Если герой ездил в этот день (маршрут длинее [0]), берём последний узел маршрута
        # как индекс подматрицы дня (1..n) и переводим в object_id. Если маршрут пустой/только депо,
        # оставляем прошлую позицию без изменений.
        open_idx = df_day_open.loc[df_day_open['day_open'] == day, 'object_id'].sort_values().values
        new_hero_end_positions = {}
        for vehicle_id in range(data["num_vehicles"]):
            route = routes[vehicle_id]
            if len(route) > 1:
                last_node = 0
                for node in reversed(route):
                    if node != 0:
                        last_node = node
                        break
                if last_node > 0:
                    new_hero_end_positions[vehicle_id] = open_idx[last_node - 1]
                else:
                    new_hero_end_positions[vehicle_id] = hero_end_positions.get(vehicle_id, 0)
            else:
                new_hero_end_positions[vehicle_id] = hero_end_positions.get(vehicle_id, 0)
        return routes, new_hero_end_positions
    else:
        print(f"No solution found for day {day}!")
        return [[] for _ in range(num_heroes)], hero_end_positions.copy()

### Преобразование маршрутов в формат решения

Ниже определена функция для преобразования маршрутов из индексов матрицы OR-Tools в реальные object_id.

In [12]:
# ============================================================================
# РЕШЕНИЕ VRP ДЛЯ ДНЯ 2
# ============================================================================

print("\n" + "=" * 60)
print("РЕШЕНИЕ ДЛЯ ДНЯ 2")
print("=" * 60)
print("Решаем VRP задачу для объектов второго дня...")
print("Каждый герой начинает с его конечной точки первого дня.\n")

routes_day2, hero_end_positions = solve_vrp_day_n(2, hero_end_positions, routes_day1)

print(f"\n{'='*60}")
print("СТАТИСТИКА ПО ДНЮ 2")
print(f"{'='*60}")
print(f"✓ Всего героев в списке: {len(routes_day2)} (все активны на все дни)")
print(f"✓ Героев с активными маршрутами: {sum(1 for r in routes_day2 if len(r) > 1)}")
print(f"✓ Героев с пустыми маршрутами: {sum(1 for r in routes_day2 if len(r) <= 1)}")

print(f"\nКонечные позиции героев после дня 2:")
print(f"{'Герой':<8} {'Позиция':<15} {'Статус'}")
print(f"{'-'*50}")
for hero_idx, pos in hero_end_positions.items():
    route = routes_day2[hero_idx]
    pos_str = f"объект {pos}" if pos > 0 else "депо"
    if len(route) > 1:
        status = "активен (что-то делал)"
    else:
        status = f"неактивен (стоял в точке {pos if pos > 0 else 'депо'})"
    print(f"  {hero_idx + 1:<6} {pos_str:<15} {status}")


РЕШЕНИЕ ДЛЯ ДНЯ 2
Решаем VRP задачу для объектов второго дня...
Каждый герой начинает с его конечной точки первого дня.


СТАТИСТИКА ПО ДНЮ 2
✓ Всего героев в списке: 20 (все активны на все дни)
✓ Героев с активными маршрутами: 17
✓ Героев с пустыми маршрутами: 3

Конечные позиции героев после дня 2:
Герой    Позиция         Статус
--------------------------------------------------
  1      объект 159      активен (что-то делал)
  2      объект 23       активен (что-то делал)
  3      объект 627      активен (что-то делал)
  4      объект 372      активен (что-то делал)
  5      объект 176      активен (что-то делал)
  6      объект 650      активен (что-то делал)
  7      объект 422      неактивен (стоял в точке 422)
  8      объект 53       активен (что-то делал)
  9      объект 548      активен (что-то делал)
  10     объект 201      активен (что-то делал)
  11     объект 268      активен (что-то делал)
  12     объект 145      активен (что-то делал)
  13     объект 319      активе

In [13]:
# ============================================================================
# РЕШЕНИЕ VRP ДЛЯ ДНЕЙ 3-7
# ============================================================================

all_routes = [routes_day1, routes_day2]

for day in range(3, 8):
    print("\n" + "=" * 60)
    print(f"РЕШЕНИЕ ДЛЯ ДНЯ {day}")
    print("=" * 60)
    print(f"Решаем VRP задачу для объектов дня {day}...")
    print(f"Каждый герой начинает с его конечной точки дня {day-1}.\n")
    previous_day_routes = all_routes[-1]
    routes_day, hero_end_positions = solve_vrp_day_n(day, hero_end_positions, previous_day_routes)
    all_routes.append(routes_day)
    active_count = sum(1 for r in routes_day if len(r) > 1)
    inactive_count = sum(1 for r in routes_day if len(r) <= 1)
    print(f"\n{'='*60}")
    print(f"СТАТИСТИКА ПО ДНЮ {day}")
    print(f"{'='*60}")
    print(f"✓ Всего героев в списке: {len(routes_day)} (все активны на все дни)")
    print(f"✓ Героев с активными маршрутами: {active_count}")
    print(f"✓ Героев с пустыми маршрутами: {inactive_count}")
    print(f"\nКонечные позиции героев после дня {day}:")
    print(f"{'Герой':<8} {'Позиция':<15} {'Статус'}")
    print(f"{'-'*50}")
    for hero_idx, pos in hero_end_positions.items():
        route = routes_day[hero_idx]
        pos_str = f"объект {pos}" if pos > 0 else "депо"
        if len(route) > 1:
            status = "активен (что-то делал)"
        else:
            status = f"неактивен (стоял в точке {pos if pos > 0 else 'депо'})"
        print(f"  {hero_idx + 1:<6} {pos_str:<15} {status}")


РЕШЕНИЕ ДЛЯ ДНЯ 3
Решаем VRP задачу для объектов дня 3...
Каждый герой начинает с его конечной точки дня 2.


СТАТИСТИКА ПО ДНЮ 3
✓ Всего героев в списке: 20 (все активны на все дни)
✓ Героев с активными маршрутами: 18
✓ Героев с пустыми маршрутами: 2

Конечные позиции героев после дня 3:
Герой    Позиция         Статус
--------------------------------------------------
  1      объект 40       активен (что-то делал)
  2      объект 23       неактивен (стоял в точке 23)
  3      объект 103      активен (что-то делал)
  4      объект 192      активен (что-то делал)
  5      объект 469      активен (что-то делал)
  6      объект 136      активен (что-то делал)
  7      объект 305      активен (что-то делал)
  8      объект 546      активен (что-то делал)
  9      объект 430      активен (что-то делал)
  10     объект 152      активен (что-то делал)
  11     объект 348      активен (что-то делал)
  12     объект 128      активен (что-то делал)
  13     объект 404      активен (что-то дел

## Формирование итогового решения

Теперь соберем все маршруты в единый формат для сохранения в CSV.

Маршруты в OR-Tools представлены индексами в отфильтрованной матрице.
Нужно преобразовать их обратно в реальные object_id.

In [14]:
def convert_route_to_object_ids(route, day):
    """
    Преобразует маршрут из индексов матрицы в реальные object_id.
    
    Args:
        route: маршрут как список индексов (0 - депо, остальные - индексы объектов)
        day: номер дня для определения доступных объектов
    
    Returns:
        Список object_id (исключая депо). Для пустых маршрутов возвращает пустой список.
    """
    # Если маршрут пустой или только депо, возвращаем пустой список
    if not route or len(route) <= 1:
        return []
    
    # Получаем список доступных объектов для дня (отсортированный по object_id)
    open_idx = df_day_open.loc[df_day_open['day_open'] == day, 'object_id'].sort_values().values
    
    # Преобразуем индексы в object_id (индекс 0 - депо, пропускаем)
    object_ids = []
    for idx in route:
        if idx == 0:  # Депо пропускаем
            continue
        # Индекс в матрице соответствует позиции в списке open_idx
        # Индекс 1 в матрице = первый объект в open_idx (индекс 0)
        if 0 < idx <= len(open_idx):
            object_ids.append(open_idx[idx - 1])
    
    return object_ids


# ============================================================================
# СОБИРАЕМ ВСЕ МАРШРУТЫ В ЕДИНЫЙ ФОРМАТ РЕШЕНИЯ
# ============================================================================

# ВАЖНО: Все герои присутствуют во всех днях, даже если у них пустые маршруты
# Пустые маршруты не добавляются в решение (герой ничего не посещал)
all_solution_routes = []

print("Преобразование маршрутов в формат решения...")
print(f"Всего дней: {len(all_routes)}")

# Проходим по всем дням и всем героям
for day_idx, day_routes in enumerate(all_routes, start=1):
    print(f"  День {day_idx}: обрабатываем {len(day_routes)} маршрутов...")
    
    for hero_idx, route in enumerate(day_routes):
        # Преобразуем маршрут из индексов матрицы в реальные object_id
        # Для пустых маршрутов вернется пустой список, и ничего не добавится в решение
        object_ids = convert_route_to_object_ids(route, day_idx)
        
        # Добавляем каждое посещение в решение
        # hero_id начинается с 1 (индекс hero_idx соответствует hero_id hero_idx+1)
        # Это важно для правильного соответствия с правилами таверны
        for obj_id in object_ids:
            all_solution_routes.append({
                'hero_id': hero_idx + 1,
                'object_id': obj_id
            })

# Создаем DataFrame с решением
solution_df = pd.DataFrame(all_solution_routes)

print(f"\n{'='*60}")
print(f"СТАТИСТИКА РЕШЕНИЯ")
print(f"{'='*60}")
print(f"✓ Всего посещений в решении: {len(solution_df)}")
print(f"✓ Уникальных объектов посещено: {solution_df['object_id'].nunique()} из {len(df_day_open)}")
print(f"✓ Процент покрытия объектов: {solution_df['object_id'].nunique() / len(df_day_open) * 100:.1f}%")
print(f"✓ Использовано героев: {solution_df['hero_id'].nunique()}")
print(f"✓ Максимальный hero_id: {solution_df['hero_id'].max()}")

# Показываем первые строки решения для проверки
print("\nПервые 20 строк решения:")
print(solution_df.head(20).to_string(index=False))

Преобразование маршрутов в формат решения...
Всего дней: 7
  День 1: обрабатываем 20 маршрутов...
  День 2: обрабатываем 20 маршрутов...
  День 3: обрабатываем 20 маршрутов...
  День 4: обрабатываем 20 маршрутов...
  День 5: обрабатываем 20 маршрутов...
  День 6: обрабатываем 20 маршрутов...
  День 7: обрабатываем 20 маршрутов...

СТАТИСТИКА РЕШЕНИЯ
✓ Всего посещений в решении: 688
✓ Уникальных объектов посещено: 688 из 700
✓ Процент покрытия объектов: 98.3%
✓ Использовано героев: 20
✓ Максимальный hero_id: 20

Первые 20 строк решения:
 hero_id  object_id
       1        506
       1        578
       1        647
       1        648
       1         61
       1         14
       2        310
       2        185
       2        196
       2        406
       2        412
       2        298
       2        271
       3         89
       3         17
       3        187
       3        457
       3         55
       3        189
       3         57


### Сохранение решения в CSV

Сохраняем решение в формате, требуемом для отправки.

In [15]:
# Сохраняем решение в CSV файл
output_filename = 'solution_baseline_294000.csv'
solution_df.to_csv(output_filename, index=False)

print(f"\nРешение сохранено в файл: {output_filename}")
print(f"Размер файла: {len(solution_df)} строк")

# ============================================================================
# ПРОВЕРКИ КОРРЕКТНОСТИ РЕШЕНИЯ
# ============================================================================
print("\n" + "=" * 60)
print("ПРОВЕРКИ КОРРЕКТНОСТИ РЕШЕНИЯ")
print("=" * 60)

# 1. Проверка диапазонов ID
print("\n1. Проверка диапазонов ID:")
hero_id_valid = solution_df['hero_id'].between(1, 100).all()
object_id_valid = solution_df['object_id'].between(1, 700).all()
print(f"   ✓ Все hero_id в диапазоне 1-100: {hero_id_valid}")
print(f"   ✓ Все object_id в диапазоне 1-700: {object_id_valid}")

if not hero_id_valid:
    invalid_heroes = solution_df[~solution_df['hero_id'].between(1, 100)]['hero_id'].unique()
    print(f"   ✗ Некорректные hero_id: {invalid_heroes}")

if not object_id_valid:
    invalid_objects = solution_df[~solution_df['object_id'].between(1, 700)]['object_id'].unique()
    print(f"   ✗ Некорректные object_id: {invalid_objects}")

# 2. Проверка уникальности объектов
print("\n2. Проверка уникальности объектов:")
duplicates = solution_df['object_id'].duplicated()
has_duplicates = duplicates.any()
print(f"   ✓ Нет дубликатов object_id: {not has_duplicates}")

if has_duplicates:
    dup_objects = solution_df[duplicates]['object_id'].unique()
    print(f"   ✗ Дублирующиеся object_id: {dup_objects[:10]}...")  # Показываем первые 10

# 3. Проверка существования объектов в данных
print("\n3. Проверка существования объектов в данных:")
valid_object_ids = set(df_day_open['object_id'].values)
solution_object_ids = set(solution_df['object_id'].values)
unknown_objects = solution_object_ids - valid_object_ids
print(f"   ✓ Все object_id существуют в данных: {len(unknown_objects) == 0}")

if unknown_objects:
    print(f"   ✗ Неизвестные object_id: {list(unknown_objects)[:10]}...")

# 4. Проверка существования героев в данных
print("\n4. Проверка существования героев в данных:")
valid_hero_ids = set(df_heroes['hero_id'].values)
solution_hero_ids = set(solution_df['hero_id'].values)
unknown_heroes = solution_hero_ids - valid_hero_ids
print(f"   ✓ Все hero_id существуют в данных: {len(unknown_heroes) == 0}")

if unknown_heroes:
    print(f"   ✗ Неизвестные hero_id: {list(unknown_heroes)}")

# 5. Проверка правил таверны (последовательный найм)
print("\n5. Проверка правил таверны (последовательный найм):")
max_hero_id = solution_df['hero_id'].max()
expected_heroes = set(range(1, max_hero_id + 1))
actual_heroes = set(solution_df['hero_id'].unique())
missing_heroes = expected_heroes - actual_heroes
print(f"   ✓ Максимальный hero_id: {max_hero_id}")
print(f"   ✓ Все герои от 1 до {max_hero_id} присутствуют: {len(missing_heroes) == 0}")

if missing_heroes:
    print(f"   ⚠ Отсутствующие hero_id (но это нормально, если они не использовались): {sorted(missing_heroes)}")

# 6. Проверка структуры данных
print("\n6. Проверка структуры данных:")
print(f"   ✓ DataFrame имеет правильные колонки: {set(solution_df.columns) == {'hero_id', 'object_id'}}")
print(f"   ✓ Нет пустых значений: {solution_df.isnull().sum().sum() == 0}")

# 7. Статистика решения
print("\n7. Статистика решения:")
unique_objects = solution_df['object_id'].nunique()
total_visits = len(solution_df)
unique_heroes = solution_df['hero_id'].nunique()
visits_per_hero = solution_df.groupby('hero_id').size()

print(f"   • Всего посещений: {total_visits}")
print(f"   • Уникальных объектов посещено: {unique_objects}")
print(f"   • Уникальных героев использовано: {unique_heroes}")
print(f"   • Среднее посещений на героя: {visits_per_hero.mean():.2f}")
print(f"   • Мин/Макс посещений на героя: {visits_per_hero.min()}/{visits_per_hero.max()}")

# 8. Подсчет метрики (приблизительный)
print("\n8. Оценка метрики VRPTW Score:")
total_reward = unique_objects * reward
total_hero_cost = max_hero_id * hero_cost
estimated_score = total_reward - total_hero_cost

print(f"   • Посещено уникальных объектов: {unique_objects}")
print(f"   • Награда за объект: {reward}")
print(f"   • Общая награда: {total_reward}")
print(f"   • Максимальный hero_id (нужно нанять всех до него): {max_hero_id}")
print(f"   • Стоимость найма героя: {hero_cost}")
print(f"   • Общая стоимость найма героев: {total_hero_cost}")
print(f"   • Оценка VRPTW Score: {estimated_score}")

# Итоговая проверка
print("\n" + "=" * 60)
all_checks_passed = (
    hero_id_valid and 
    object_id_valid and 
    not has_duplicates and 
    len(unknown_objects) == 0 and 
    len(unknown_heroes) == 0
)

if all_checks_passed:
    print("✓ ВСЕ ПРОВЕРКИ ПРОЙДЕНЫ УСПЕШНО!")
else:
    print("⚠ НЕКОТОРЫЕ ПРОВЕРКИ НЕ ПРОЙДЕНЫ. Проверьте вывод выше.")
print("=" * 60)


Решение сохранено в файл: solution_baseline_294000.csv
Размер файла: 688 строк

ПРОВЕРКИ КОРРЕКТНОСТИ РЕШЕНИЯ

1. Проверка диапазонов ID:
   ✓ Все hero_id в диапазоне 1-100: True
   ✓ Все object_id в диапазоне 1-700: True

2. Проверка уникальности объектов:
   ✓ Нет дубликатов object_id: True

3. Проверка существования объектов в данных:
   ✓ Все object_id существуют в данных: True

4. Проверка существования героев в данных:
   ✓ Все hero_id существуют в данных: True

5. Проверка правил таверны (последовательный найм):
   ✓ Максимальный hero_id: 20
   ✓ Все герои от 1 до 20 присутствуют: True

6. Проверка структуры данных:
   ✓ DataFrame имеет правильные колонки: True
   ✓ Нет пустых значений: True

7. Статистика решения:
   • Всего посещений: 688
   • Уникальных объектов посещено: 688
   • Уникальных героев использовано: 20
   • Среднее посещений на героя: 34.40
   • Мин/Макс посещений на героя: 24/45

8. Оценка метрики VRPTW Score:
   • Посещено уникальных объектов: 688
   • Награда